# Modelagem de classificação de clientes com dados da saúde suplementar

Vamos usar os [dados obtidos anteriormente](https://github.com/VFLins/hospitalar_data_ans/blob/main/notebooks/collect_routine_ans.ipynb) e o conhecimento que obtivemos na sua [análise exploratória](https://vflins.github.io/hospitalar_data_ans/pages/eda.html) para produzir uma solução de dados com valor de negócio para empresas do **setor de operação de saúde.**

A proposta deste modelo é produzir um indicador do *"custo potencial de aquisição"* de um cliente. Esta solução pode facilitar grandemente a precificação dos produtos de saúde, além disso, neste processo podemos acabar aprendendo mais sobre os diferentes tipos de perfis de clientes e suas necessidades!

In [2]:
# imports

from pyspark.sql import SparkSession
from getpass import getpass

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [4]:
# variaveis de conexao

drivername = "oracle.jdbc.OracleDriver"
WALLET_PLACE = "/home/user/Downloads/wallet_demodb"
URL = f"jdbc:oracle:thin:@demodb_medium?TNS_ADMIN={WALLET_PLACE}"
USR = "ADMIN"
PWD = getpass("Insira a senha de administrador do banco de dados: ")

Insira a senha de administrador do banco de dados:  ········


In [3]:
# conjuntos de dados

tablename = "HOSP_CONS"
query = f"SELECT * FROM {tablename}"

cons_df = spark.read.format("jdbc")\
  .option("driver", drivername)\
  .option("query", query)\
  .option("url", URL)\
  .option("user", USR)\
  .option("password", PWD)\
  .load()


tablename = "HOSP_DET"
query = f"SELECT * FROM {tablename}"

det_df = spark.read.format("jdbc")\
  .option("driver", drivername)\
  .option("query", query)\
  .option("url", URL)\
  .option("user", USR)\
  .option("password", PWD)\
  .load()

NameError: name 'drivername' is not defined

# 1 Criando um *target*

Vamos abordar este problema usando um modelo de classificação. Para isto vamos precisar de um variável target binária que separe os clientes entre "alto custo" e "baixo custo". Usaremos a probabilidade estimada de um cliente pertencer a um grupo ou à outro como o *score* que indica seu custo potencial.

In [7]:
print(f"Consolidada:\n{cons_df.schema.names}", f"Detalhada:\n{det_df.schema.names}", sep="\n\n")

Consolidada:
['ID_EVENTO_ATENCAO_SAUDE', 'ID_PLANO', 'FAIXA_ETARIA', 'SEXO', 'CD_MUNICIPIO_BENEFICIARIO', 'PORTE', 'CD_MODALIDADE', 'NM_MODALIDADE', 'CD_MUNICIPIO_PRESTADOR', 'UF_PRESTADOR', 'TEMPO_DE_PERMANENCIA', 'ANO_MES_EVENTO', 'CD_CARATER_ATENDIMENTO', 'CD_TIPO_INTERNACAO', 'CD_REGIME_INTERNACAO', 'CD_MOTIVO_SAIDA', 'CID_1', 'CID_2', 'CID_3', 'CID_4', 'QT_DIARIA_ACOMPANHANTE', 'QT_DIARIA_UTI', 'IND_ACIDENTE_DOENCA', 'LG_VALOR_PREESTABELECIDO']

Detalhada:
['ID_EVENTO_ATENCAO_SAUDE', 'UF_PRESTADOR', 'TEMPO_DE_PERMANENCIA', 'ANO_MES_EVENTO', 'CD_PROCEDIMENTO', 'CD_TABELA_REFERENCIA', 'QT_ITEM_EVENTO_INFORMADO', 'VL_ITEM_EVENTO_INFORMADO', 'VL_ITEM_PAGO_FORNECEDOR', 'IND_PACOTE', 'IND_TABELA_PROPRIA']


## 1.1 A especificidade de cada conjunto de dados

Para obter o custo de um cliente, seria ideal ter um conjunto de dados que representa um cliente único em cada linha, em vez disto, temos:

- A nossa base de dados **consolidadda** apresentando um atendimento médico para cada linha
- A base de dados **detalhada** apresentando um procedimento do atendimento médico para cada linha

Podemos perceber que as variáveis que descrevem o cliente (exemplo: `"FAIXA_ETARIA"`, `"SEXO"`, etc.) estão presentes na tabela *consolidada*. Para obter um conjunto no formato que precisamos para modelagem, será necessário agregar algumas características dos consumidores presentes na tabela consolidada.

Depois que tivermos como "sinalizar" quais são os clientes únicos presentes na tabela *consolidada*, podemos extrair mais informações sobre seus respctivos eventos hospitalares na tabela *detalhada*.